In [1]:
import re

from selenium.webdriver import Chrome
from json import load,dump

In [2]:
bot = Chrome()

In [3]:
base_url = 'https://keras.io/api/layers'

In [4]:
bot.get(base_url)

In [5]:
groups = [ group.text.lower().replace("-","").replace(" ","-") for group in bot.find_elements_by_tag_name("h3")[5:]]

In [ ]:
elem = None

config = {}

for group in groups:
    bot.get(base_url)
    elem = bot.find_element_by_id(group)
    elem.find_element_by_tag_name("a").click()
    
    group_url = bot.current_url
    group_name = bot.find_element_by_id(group).text
    
    layer_urls = [ (a.text,a.get_property("href")) for a in bot.find_element_by_tag_name("ul").find_elements_by_tag_name("a")]
    group_layers = []
    for name,url in layer_urls:
        bot.get(url)
        layer_name = re.sub("( layer)|( object)|( layers)","",name)
        code = bot.find_element_by_tag_name("pre")
        args = dict([
            i.strip()
            .replace("\"","")
            .replace(",","")
            .split("=") if "=" in i else (i.strip().replace("\"","").replace(",",""),"REQUIRED")
            for i in 
            code.text.split("\n")[1:-1]
            if 'kwargs' not in i
        ])
        group_layers.append(dict(
            name=layer_name,
            args=args,
            doc=url
        ))
        
    config[group] = dict(
        name=group_name,
        layers=group_layers,
        doc=group_url
    )

In [ ]:
with open("../data/layer_config.json","w+") as file:
    dump(config,file)